In [1]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim import Adam
import torch.nn.functional as F

import csv
from skimage import io

from PIL import Image
import pandas as pd

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import matplotlib.pyplot as plt
import time
import os
import copy

import sys
sys.path.append('JupyterProjects/EYE-SEA/DataSets/MARVEL_code')

import import_ipynb
import Dataset_loader_MARVEL
import ResNetCaps

verbose = False
USE_CUDA = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

CIFAR10_USE = True
MARVEL_USE = False

importing Jupyter notebook from Dataset_loader_MARVEL.ipynb
importing Jupyter notebook from ResNetCaps.ipynb


In [2]:
dataset_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),        
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])


batch_size = 32


if CIFAR10_USE: 
    NUM_CLASSES = 10
    print("CIFAR10")
    image_datasets = {'train': datasets.CIFAR10('../data', train=True, download=True, transform=dataset_transform),'val': datasets.CIFAR10('../data', train=False, download=True, transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")

    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")

if MARVEL_USE: 
    NUM_CLASSES = 26
    print("MARVEL")
    dat_file = "/home/rita/JupyterProjects/EYE-SEA/DataSets/marveldataset2016-master/FINAL.dat"

    image_datasets = {'train': MARVEL_dataset(dat_file,train = True,transform=dataset_transform),'val': MARVEL_dataset(dat_file,train = False,transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")

    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

CIFAR10
Files already downloaded and verified
Files already downloaded and verified
Initializing Datasets and Dataloaders...
Initializing Datasets and Dataloaders...


In [3]:
def shift_left_once(A):
    a =  []
    AD = A.detach().cpu().numpy()
    temp = AD[0,:]

    for index in range(1,len(AD)):
        a = np.concatenate([a,AD[index]])
    a = np.concatenate([a,temp])
    return ((torch.tensor(a).view(len(a),1)).float()).to(device)

def shift_left(lst, n):
    assert (n >= 0), "n should be non-negative integer"
    for _ in range(n):
        shift_left_once(lst)

def circulant(A):
    r = A.size()[0]
    circ_list_a = []
    circ_list_a.append(A)
    
    a = shift_left_once(A)
    for i in range(r-1):   
        circ_list_a.append(a)      
        a = shift_left_once(a)

    
    circ = torch.cat(circ_list_a,0)
    circ = circ.view([r,r])
    if verbose: print("circular matrix {}".format(circ))
    return circ.to(device)
        
        
    

In [4]:
class Interaction(nn.Module):
    def __init__(self,NUM_CLASSES):
        super(Interaction, self).__init__()
        self.modelCaps1 = ResNetCaps.ResNetCaps(NUM_CLASSES)
        self.modelCaps2 = ResNetCaps.ResNetCaps(NUM_CLASSES)
        
        self.W1 = torch.randn(NUM_CLASSES,NUM_CLASSES)
        self.W1.requires_grad = True
        
        self.W2 = torch.randn(NUM_CLASSES,NUM_CLASSES)
        self.W2.requires_grad = True
        
        self.modelLin = nn.Linear(NUM_CLASSES, NUM_CLASSES)
        self.NClass = NUM_CLASSES

    def cuda(self, device=None):
        self = super().cuda(device)
        self.W1 = self.W1.cuda(device)
        self.W2 = self.W2.cuda(device)
        return self 
        
    def forward(self,inputs):
        digit1,_ = self.modelCaps1(inputs)
        digit2,_ = self.modelCaps2(inputs)
        
        output = F.softmax(self.modelLin(self.interaction(digit1,digit2)),dim=1)
        return output
        
    def interaction(self,x,y):
        
        self.x = x
        self.y = y
         
        Z_list = []
        labels_size = x.size()[1]
        batch = min(x.size()[0],batch_size)
        for i in range(batch):
        
            xi = torch.sum(x[i,:,:,0],dim=1)
            yi = torch.sum(y[i,:,:,0],dim=1)
            
            xi = xi.view(self.NClass,1)
            yi = yi.view(self.NClass,1)

            V = torch.mm(self.W1,xi)   #< questo fa batch-matrix con matrix ?
            C = torch.mm(self.W2,yi)

            #CIRCOLANTE
            A = circulant(V)
            B = circulant(C)

            #INTERACTION MOMENT
            F = torch.mm(B,V)
            G = torch.mm(A,C)
            
            M = torch.add(F,G)

            Z_list.append(M)
            
        Z = torch.cat(Z_list,0)
        if verbose : print(Z.size())
        Z = Z.view(batch,labels_size)
        
        return Z   
        
    

In [ ]:
model = Interaction(NUM_CLASSES)
#model = model.to(device)
model = model.cuda(device)

#optimizers
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr = 0.001)
n_epochs = 30

#train
start = time.time()
#batch_id = 100
#inputs, labels = next(iter(dataloaders['train']))
accuracy_train = []
loss_train = []


for epoch in range(n_epochs): 
    model.train() 
    train_loss = 0
    train_accuracy = 0

    print('epoch {}:{}'.format(epoch+1, n_epochs)) 
    for batch_id, (inputs, labels) in enumerate(dataloaders['train']):
        labels =torch.eye(NUM_CLASSES).index_select(dim=0, index=labels)
        inputs, labels = Variable(inputs), Variable(labels)
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        _,label = torch.max(labels, 1)

        loss = criterion(outputs, label.long())
        loss.backward()
        optimizer.step()

        train_loss += loss.data[0]
        train_accuracy += (sum(np.argmax(outputs.data.cpu().numpy(), 1) == np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))

        if batch_id % 100 == 0:
            print("train accuracy:", sum(np.argmax(outputs.data.cpu().numpy(), 1) == 
                                   np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))
            if verbose: print("masked {}".format(np.argmax(masked.data.cpu().numpy(), 1)))
            if verbose: print("labels {}".format(np.argmax(labels.data.cpu().numpy(), 1)))
        #                batch_accuracy.append(sum(np.argmax(preds.data.cpu().numpy(), 1) == 
        #                                       np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))
    accuracy_train.append(train_accuracy/len(dataloaders['train']))
    loss_train.append(train_loss/len(dataloaders['train']))

end = time.time()
print("Training time execution {}".format(end-start))
print("Loss value for training phase: {}".format(train_loss / len(dataloaders['train'])))
print("Accuracy value for training phase: {}".format(train_accuracy / len(dataloaders['train'])))

epoch 1:30


ResNetCaps.ipynb:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  "from torch.autograd import Variable\n",
ResNetCaps.ipynb:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  "USE_CUDA = True\n",
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


train accuracy: 0.03125
train accuracy: 0.0625
train accuracy: 0.1875
train accuracy: 0.3125
train accuracy: 0.4375
train accuracy: 0.46875
train accuracy: 0.46875
train accuracy: 0.46875
train accuracy: 0.625
train accuracy: 0.46875
train accuracy: 0.65625
train accuracy: 0.59375
train accuracy: 0.5
train accuracy: 0.625
train accuracy: 0.65625
train accuracy: 0.625
epoch 2:30
train accuracy: 0.59375
train accuracy: 0.6875
train accuracy: 0.53125
train accuracy: 0.46875
train accuracy: 0.5625
train accuracy: 0.625
train accuracy: 0.5625
train accuracy: 0.40625
train accuracy: 0.625
train accuracy: 0.5625
train accuracy: 0.65625
train accuracy: 0.625
train accuracy: 0.5
train accuracy: 0.65625
train accuracy: 0.5
train accuracy: 0.625
epoch 3:30
train accuracy: 0.5625
train accuracy: 0.5
train accuracy: 0.59375
train accuracy: 0.6875
train accuracy: 0.5
train accuracy: 0.71875
train accuracy: 0.53125
train accuracy: 0.71875
train accuracy: 0.59375
train accuracy: 0.625
train accuracy: 

In [ ]:
epochs = np.arange(1,n_epochs+1)
plt.plot(epochs, loss_train, color='g')
plt.plot(epochs, accuracy_train, color='orange')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training phase')
plt.show() 